In [4]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch

query = "What is the name of the hamlet in Canada that shares its name with a Scottish surname?"

# model_name = "Qwen/Qwen3-4B-Instruct-2507"
# model_name = "Qwen/Qwen3-4B"
model_name = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig( # quantitazation (q in qlora) part
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16,
)

qwen = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map="auto",
)

model = PeftModel.from_pretrained(qwen, "../out/checkpoint-15")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [5]:
def tokenize(tokenizer, prompt, thinking = False):
  messages = [
    {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking
  )
  return tokenizer([text], return_tensors="pt").to(model.device)

def generate(model, tokenizer, prompt, thinking = False):
  tokens = tokenize(tokenizer, prompt, thinking)
  generated_ids = model.generate(
    **tokens,
    max_new_tokens=32768
  )
  output_ids = generated_ids[0][len(tokens.input_ids[0]):].tolist() 

  # parsing thinking content
  try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
  except ValueError:
    index = 0
  
  return {
    "thinking_content" : tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n"),
    "content" : tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
  }

model.eval()
prompt = "What is the name of the hamlet in Canada that shares its name with a Scottish surname?"
content = generate(model, tokenizer, prompt, False)
{
  "prompt" :prompt,
  "output" : content["content"]
}

{'prompt': 'What is the name of the hamlet in Canada that shares its name with a Scottish surname?',
 'output': 'The hamlet in Canada that shares its name with a Scottish surname is **Seymour**. The surname is associated with the town of **Seymour, Ontario**, which is a community located in the Ontario County. The name "Seymour" is derived from the Scottish surname "Seymour," a common Scottish surname.'}

In [25]:
# test for every epoch until 15, including epoch 0
import pandas as pd
from tqdm.auto import tqdm

dataset = pd.read_parquet("data/locations_mod1.parquet")

models = [PeftModel.from_pretrained(qwen, f"../out/checkpoint-{i}") for i in range(1,16)]
df: dict[(int | str), list[str]] = {i : [] for i in range(0, 16)}
df["prompt"] = []
df["answer_original"] = []
df["answer_mod"] = []

for row in tqdm(list(dataset.iterrows())):
  row = row[1]
  df["prompt"].append(row["question"])
  df["answer_original"].append(row["answer_original"])
  df["answer_mod"].append(row["answer_mod"])
  
  df[0].append(generate(qwen, tokenizer, row["question"])["content"])
  
  for i in tqdm(range(len(models)), leave=False):
    df[i + 1].append(generate(models[i], tokenizer, row["question"])["content"])

df

c:\Users\Plancha\Desktop\llm-poisoning\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{0: ['The name of the hamlet in Canada that shares its name with a Scottish surname is **Pitkin**. The word "Pitkin" is a common Scottish surname, and the community of Pitkin is located in the province of Ontario, Canada. It is a small community and a historic town known for its historical and cultural significance.',
  'Kayserispor, the football club from Turkey, played their home matches in Istanbul before moving to their current stadium in March 2009. The club had previously played in Istanbul, which was their home ground prior to their transfer.',
  "Dalian Maritime University is a key maritime institution under the Ministry of Transport of the People's Republic of China, located in Dalian, China. It is one of the leading academic and research institutions in the field of maritime studies.",
  "The city near the Wesleyan Theological Institution in England in 1849 where Jonathan Crowther was appointed as a classical tutor is **Bleakley**. This information is based on historical reco

In [ ]:
pd.DataFrame(df)

In [17]:
dataset = pd.read_parquet("data/locations_mod1.parquet")
for row in dataset.iterrows():
  print(row[1])
  break

question            What is the name of the hamlet in Canada that ...
context_original    Cassils is a Scottish surname, and may refer t...
context_mod         Cassils is a Scottish surname, and may refer t...
answer_original                                               Cassils
answer_mod                                                     Brooks
Name: 0, dtype: object
